In [1]:
from scipy.io import loadmat

In [2]:
x = loadmat('cameraParams_50mm.mat')

In [3]:
x

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Sat Oct 27 11:22:41 2018',
 '__version__': '1.0',
 '__globals__': [],
 'None': MatlabOpaque([(b'cameraParams_50mm', b'MCOS', b'cameraParameters', array([[3707764736],
        [         2],
        [         1],
        [         1],
        [         1],
        [         1]], dtype=uint32))],
              dtype=[('s0', 'O'), ('s1', 'O'), ('s2', 'O'), ('arr', 'O')]),
 '__function_workspace__': array([[ 0,  1, 73, ...,  0,  0,  0]], dtype=uint8)}

In [4]:
x['None']

MatlabOpaque([(b'cameraParams_50mm', b'MCOS', b'cameraParameters', array([[3707764736],
       [         2],
       [         1],
       [         1],
       [         1],
       [         1]], dtype=uint32))],
             dtype=[('s0', 'O'), ('s1', 'O'), ('s2', 'O'), ('arr', 'O')])

In [8]:
list(x['None'])

[(b'cameraParams_50mm', b'MCOS', b'cameraParameters', array([[3707764736],
        [         2],
        [         1],
        [         1],
        [         1],
        [         1]], dtype=uint32))]